In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("pysparkdf").getOrCreate()
spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=9c656efadecc5559e35ac8d306a445fb4ddf5c4230f1a4114e1960b6d292e38f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
df = spark.read.format('csv').option('header','true').load('Weather_Leeds.csv')
df = df.drop('_c0')

In [3]:
df.show()

+-------------------+-----+----------+-------+--------+---------+-----------+--------+--------+--------+--------+----------+----------+-----------+-------+-------+-------+-------+----------+-------------+--------------------+----------+-------------------+-------------------+-----------------+------+-----+-----+-----+-----+----+-----+-----+
|          Timestamp| City|Population|Country|Latitude|Longitude|Temperature|Temp_min|Temp_max|Pressure|Humidity|Visibility|Wind_speed|Wind_degree|Rain_1h|Rain_3h|Snow_1h|Snow_3h|Weather_ID|Weather_group|         Description|Cloudiness|            Sunrise|             Sunset|Air_quality_index|    CO|   NO|  NO2|   O3|  SO2| NH3|PM2_5| PM10|
+-------------------+-----+----------+-------+--------+---------+-----------+--------+--------+--------+--------+----------+----------+-----------+-------+-------+-------+-------+----------+-------------+--------------------+----------+-------------------+-------------------+-----------------+------+-----+-----+-

In [4]:
from pyspark.sql.functions import *

The rows were successfuly imported, but all as strings, so this block was needed to carry out operations

In [5]:
df = df.withColumn('Timestamp', df.Timestamp.cast('timestamp'))
df = df.withColumn('Population', df.Population.cast('int'))
df = df.withColumn('Latitude', df.Latitude.cast('float'))
df = df.withColumn('Longitude', df.Longitude.cast('float'))
df = df.withColumn('Temperature', df.Temperature.cast('float'))
df = df.withColumn('Temp_min', df.Temp_min.cast('float'))
df = df.withColumn('Temp_max', df.Temp_max.cast('float'))
df = df.withColumn('Pressure', df.Pressure.cast('int'))
df = df.withColumn('Humidity', df.Humidity.cast('int'))
df = df.withColumn('Visibility', df.Visibility.cast('int'))
df = df.withColumn('Wind_speed', df.Wind_speed.cast('float'))
df = df.withColumn('Wind_degree', df.Wind_degree.cast('float'))
df = df.withColumn('Rain_1h', df.Rain_1h.cast('int'))
df = df.withColumn('Rain_3h', df.Rain_1h.cast('int'))
df = df.withColumn('Snow_1h', df.Snow_1h.cast('int'))
df = df.withColumn('Snow_3h', df.Snow_3h.cast('int'))
df = df.withColumn('Weather_ID', df.Weather_ID.cast('int'))
df = df.withColumn('Cloudiness', df.Cloudiness.cast('int'))
df = df.withColumn('Sunrise', df.Sunrise.cast('timestamp'))
df = df.withColumn('Sunset', df.Sunset.cast('timestamp'))
df = df.withColumn('Air_quality_index', df.Air_quality_index.cast('int'))
df = df.withColumn('CO', df.CO.cast('float'))
df = df.withColumn('NO', df.NO.cast('float'))
df = df.withColumn('NO2', df.NO2.cast('float'))
df = df.withColumn('O3', df.O3.cast('float'))
df = df.withColumn('SO2', df.SO2.cast('float'))
df = df.withColumn('NH3', df.NH3.cast('float'))
df = df.withColumn('PM2_5', df.PM2_5.cast('float'))
df = df.withColumn('PM10', df.PM10.cast('float'))

In [6]:
temps = df.select('Temperature')
median = np.median(temps.collect())
print(median)

6.260000228881836


In [8]:
temps2 = df.select('Timestamp','Temperature','CO')
temps2.show()

+-------------------+-----------+------+
|          Timestamp|Temperature|    CO|
+-------------------+-----------+------+
|2023-12-11 20:00:02|       5.98|380.52|
|2023-12-11 20:30:04|        6.3|407.22|
|2023-12-11 21:00:03|       6.49|407.22|
|2023-12-11 21:30:03|        6.0|433.92|
|2023-12-11 22:00:03|       6.24|433.92|
|2023-12-11 22:30:03|        6.0| 440.6|
|2023-12-11 23:00:03|        6.0| 440.6|
|2023-12-11 23:30:03|       5.77|420.57|
|2023-12-12 00:00:03|       5.48|420.57|
|2023-12-12 00:30:03|       5.39|360.49|
|2023-12-12 01:00:03|       5.26|360.49|
|2023-12-12 01:30:02|       5.39|330.45|
|2023-12-12 02:00:02|       5.43|330.45|
|2023-12-12 02:30:02|       5.52|310.42|
|2023-12-12 03:00:02|        5.6|310.42|
|2023-12-12 03:30:02|       5.83|283.72|
|2023-12-12 04:00:02|       5.96|283.72|
|2023-12-12 04:30:02|       6.09|260.35|
|2023-12-12 05:00:02|       6.14|260.35|
|2023-12-12 05:30:02|       6.12|243.66|
+-------------------+-----------+------+
only showing top

The temperature and CO levels do not appear to directly correlate - as the temperature dips, so does the CO concentration, but when temperature begins to climb again, the CO concentration does not

In [37]:
poll = df.select('Timestamp','Air_quality_index','CO','NO','NO2','O3','SO2','NH3','PM2_5','PM10')
poll = poll.sort(col('CO').asc())
poll.show()

+-------------------+-----------------+------+----+-----+-----+----+----+-----+----+
|          Timestamp|Air_quality_index|    CO|  NO|  NO2|   O3| SO2| NH3|PM2_5|PM10|
+-------------------+-----------------+------+----+-----+-----+----+----+-----+----+
|2023-12-13 02:30:03|                1|223.64|0.01| 6.08|31.47|1.24|0.55| 1.34|1.86|
|2023-12-13 03:00:03|                1|223.64|0.01| 6.08|31.47|1.24|0.55| 1.34|1.86|
|2023-12-13 01:30:03|                1|226.97|0.01| 7.11|32.19|1.22|0.52| 2.17| 2.8|
|2023-12-13 02:00:04|                1|226.97|0.01| 7.11|32.19|1.22|0.52| 2.17| 2.8|
|2023-12-13 03:30:02|                1|230.31|0.01| 7.03|28.61|1.25|0.48| 1.09| 1.5|
|2023-12-13 04:00:02|                1|230.31|0.01| 7.03|28.61|1.25|0.48| 1.09| 1.5|
|2023-12-16 00:30:02|                2|230.31| 0.0| 4.37|67.95|1.68|0.21|  1.5|2.88|
|2023-12-13 00:30:04|                1|236.99|0.01|11.48|27.18|1.42| 0.4| 3.88|4.31|
|2023-12-13 01:00:03|                1|236.99|0.01|11.48|27.18|1.

Sorting by the concentration of CO shows that it is not the sole contributor to the air quality index (which is explained in the documentation, this is just an example to show that in Spark)

In [34]:
readings = df.select('Timestamp','Temperature','Wind_speed','Visibility','Weather_group','Description')
readings_when_raining = readings.withColumn('Is_it_raining?',when(readings.Weather_group == 'Rain', 'Yes').when(readings.Weather_group == 'Drizzle','Sort of').otherwise('No'))
readings_when_raining.show()

+-------------------+-----------+----------+----------+-------------+--------------------+--------------+
|          Timestamp|Temperature|Wind_speed|Visibility|Weather_group|         Description|Is_it_raining?|
+-------------------+-----------+----------+----------+-------------+--------------------+--------------+
|2023-12-11 20:00:02|       5.98|      1.54|     10000|       Clouds|       broken clouds|            No|
|2023-12-11 20:30:04|        6.3|      1.54|      6000|       Clouds|       broken clouds|            No|
|2023-12-11 21:00:03|       6.49|      1.54|     10000|      Drizzle|light intensity d...|       Sort of|
|2023-12-11 21:30:03|        6.0|      1.54|     10000|      Drizzle|light intensity d...|       Sort of|
|2023-12-11 22:00:03|       6.24|      1.54|      3000|      Drizzle|light intensity d...|       Sort of|
|2023-12-11 22:30:03|        6.0|      1.54|       500|          Fog|                 fog|            No|
|2023-12-11 23:00:03|        6.0|      2.06|  

Added an additional column to highlight when it is raining

In [32]:
reading_only_rain = readings.filter(readings.Weather_group == 'Rain')
reading_only_rain.show()

+-------------------+-----------+----------+----------+-------------+--------------------+
|          Timestamp|Temperature|Wind_speed|Visibility|Weather_group|         Description|
+-------------------+-----------+----------+----------+-------------+--------------------+
|2023-12-12 02:30:02|       5.52|      2.82|     10000|         Rain|       moderate rain|
|2023-12-12 03:00:02|        5.6|      0.45|     10000|         Rain|          light rain|
|2023-12-12 03:30:02|       5.83|      0.45|      3090|         Rain|heavy intensity rain|
|2023-12-12 04:00:02|       5.96|      0.45|      3090|         Rain|heavy intensity rain|
|2023-12-12 04:30:02|       6.09|      0.45|      3092|         Rain|          light rain|
|2023-12-14 05:00:02|       1.87|      0.45|     10000|         Rain|          light rain|
+-------------------+-----------+----------+----------+-------------+--------------------+



Similar to above, but only selecting rows when it is raining. You could also match the description to be more precise.